<h2>Data Wrangling</h2>

The goal of this step is mainly to convert the source Leica file into individual 3D tiff files. For some data, the current python reader cannot load the channel dimenstion and the Z dimension correctly. So, we have to re-order the data. We only need such dimension fix on the very first dataset. 

Parameter selection

<b>Use dimension fix</b>:<br>
    For future data, just try to run without dimension fix. 
    If the output is not correct, then re-run with dimension fix.

<b>Channels</b>:<br>
        specific chanel index. Currently, we assume the second channel 
        (index=1) is CD31 and the third channel (index=2) is Col IV 
        (note: index is zero-based)
        please provide comma-separated indices by default we use 1,2.
        
<b>LIF files path</b>:<br>
        you file path to data /path/to/string_vessel_data/
    
<b>Output path</b>:<br>
        specific where you want to save the splitted files
        /path/to/output_folder/
        
    
    
    
        
   
    

In [1]:
from core.image_processor import run_image_processing_menu
run_image_processing_menu()

Loading LIF Image Processing Configuration...


Checkbox(value=False, description='Use dimension fix?', indent=False)

Text(value='1, 2', continuous_update=False, description='Channels:')

Text(value='/path/to/string_vessel_data/', continuous_update=False, description='LIF files path:', layout=Layo…

Text(value='/path/to/output_folder/', continuous_update=False, description='Output path:', layout=Layout(width…

Button(button_style='success', description='Run Processing', icon='play', style=ButtonStyle(), tooltip='Click …

Output()